# __Facial Expression Recognition__

The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centered and occupies about the same amount of space in each image. The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories 

0. Angry, 
1. Disgust, 
2. Fear, 
3. Happy, 
4. Sad, 
5. Surprise, 
6. Neutral

Data Link: https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data

I will test the facial expressio on trained model which I have trained in emotion_recognition notebook.

In [10]:
# this program will load the model trained in emotion_recognition program
# we will test the facial emotion with the video stream
# import the dependencies
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

In [11]:
#load model
model = model_from_json(open("emotion_det_cnn.json", "r").read())
#load weights
model.load_weights('emotion_det_cnn.h5')

In [12]:
# load the harr file which will detect face
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [13]:
# open the web cam to stream video
cap=cv2.VideoCapture(0)

while True:
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    if not ret:
        continue
    # convert the test image into a gray scale image
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    # detect the face in this grau image
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, scaleFactor=1.5, minNeighbors=5)


    for (x,y,w,h) in faces_detected:
        # make a rectangle around the detected face in video stream
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        # pre process the image to fit with model
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        # predict the emotion on the trained model
        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index] # get the label of emotion

        # put a text on the image with label
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (640, 480))
    cv2.imshow('Facial emotion analysis ',resized_img)

    if cv2.waitKey(27) & 0xFF == ord('q'):
        break

# when everything is done release the capture
cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>